# Демонстрация пайплайна обработки платежных счетов

Этот ноутбук демонстрирует работу пайплайна извлечения данных из PDF файлов платежных счетов.


## Импорт библиотек


In [ ]:
import sys
import os
from pathlib import Path
import json
from io import BytesIO

# Добавляем путь к backend
sys.path.append(str(Path('../backend').absolute()))

from app.utils.pdf_parser import PDFParser
from app.utils.ocr import OCRProcessor
from app.services.extractor import PaymentInvoiceExtractor
from app.config import settings


## Загрузка тестового PDF файла


In [ ]:
# Укажите путь к тестовому PDF файлу
pdf_path = "../test_data/sample_invoice.pdf"  # Замените на путь к вашему файлу

if os.path.exists(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf_bytes = BytesIO(f.read())
    print(f"✓ Файл загружен: {pdf_path}")
    print(f"  Размер: {len(pdf_bytes.getvalue()) / 1024:.2f} КБ")
else:
    print(f"⚠ Файл не найден: {pdf_path}")
    print("Пожалуйста, укажите путь к тестовому PDF файлу")


## Шаг 1: Извлечение текста из PDF


In [ ]:
# Инициализация парсера
parser = PDFParser()

# Извлечение текста
pdf_bytes.seek(0)
text = parser.extract_text(pdf_bytes)

print(f"Извлечено символов: {len(text)}")
print(f"Извлечено строк: {len(text.splitlines())}")
print("\n" + "="*80)
print("Первые 500 символов извлеченного текста:")
print("="*80)
print(text[:500])


## Шаг 2: OCR обработка (если нужно)


In [ ]:
# Проверка доступности OCR
try:
    ocr_processor = OCRProcessor()
    print("✓ OCR доступен")
    
    # Если обычный парсинг не дал результатов, используем OCR
    if not text or len(text.strip()) < 100:
        print("\nТекст не извлечен обычным парсингом, используем OCR...")
        pdf_bytes.seek(0)
        ocr_text = ocr_processor.extract_text_from_pdf_images(pdf_bytes)
        print(f"OCR извлек символов: {len(ocr_text)}")
        print("\nПервые 500 символов из OCR:")
        print(ocr_text[:500])
    else:
        print("\nОбычный парсинг успешен, OCR не требуется")
except ImportError as e:
    print(f"⚠ OCR недоступен: {e}")


## Шаг 3: Извлечение структурированных данных


In [ ]:
# Инициализация экстрактора
extractor = PaymentInvoiceExtractor()

# Извлечение данных
pdf_bytes.seek(0)
result = extractor.extract(pdf_bytes, use_ocr=False)

print("Результат извлечения данных:")
print("="*80)
print(json.dumps(result, ensure_ascii=False, indent=2))
